In [1]:
# Import necessary modules from your files
import os
import subprocess
import numpy as np

from tqdm.notebook import tqdm

from searchclient.agent_types.classic import * 

# Import all action classes (used for hardcoding solutions) and actions libraries
from searchclient.domains.hospital.actions import (
    NoOpAction, MoveAction, PushAction, PullAction, AnyAction, DEFAULT_MAPF_ACTION_LIBRARY, DEFAULT_HOSPITAL_ACTION_LIBRARY
)

# Import state, goal description and level classes for the MAvis hospital environment
from searchclient.domains.hospital.state import HospitalState
from searchclient.domains.hospital.goal_description import HospitalGoalDescription
from searchclient.domains.hospital.level import HospitalLevel

# Import the Graph-Search algorithm
from searchclient.search_algorithms.graph_search import graph_search

# Import the different search strategies for both uninformed and informed search
from searchclient.strategies.bfs import FrontierBFS
from searchclient.strategies.dfs import FrontierDFS
from searchclient.strategies.bestfirst import FrontierBestFirst, FrontierGreedy, FrontierAStar

# Import heuristic classes, to be used in informed search methods
from searchclient.domains.hospital.heuristics import (
    HospitalZeroHeuristic, HospitalGoalCountHeuristics, HospitalAdvancedHeuristics
)

# Ensure the environment is set up properly
print("Modules imported successfully.")

Modules imported successfully.


In [2]:
# Function to load a level file
def load_level_file_from_path(path):
    with open(path, "r") as f:
        lines = f.readlines()
        lines = list(map(lambda line: line.strip(), lines))
        return lines
  
# Example usage: load_level('path_to_level_file.lvl')
level_path = "levels/TwoAgentsDebug.lvl"
level_lines = load_level_file_from_path(level_path)
level = HospitalLevel.parse_level_lines(level_lines)

# We can access the initial state of the level using the following code
initial_state = HospitalState(level, level.initial_agent_positions, level.initial_box_positions)

# We can access the goal description of the level using the following code
goal_description = HospitalGoalDescription(level, level.box_goals + level.agent_goals)

print('The initial state of the level is:')
print(initial_state)

print('\nThe goal description of the level is:')
print(goal_description) # which tells us where the level objects (like boxes and agents) should be placed to satisfy the goal
print('\nSo agent zero starts at {} and satisfies the goal at {}'.format(level.initial_agent_positions[0][0], goal_description.agent_goals[0][0]))

The initial state of the level is:
++++++++
+    2 +
+      +
+ 0    +
+      +
+ 1    +
+      +
++++++++

The goal description of the level is:
((5, 5), '0', True) and ((6, 3), '1', True)

So agent zero starts at (3, 2) and satisfies the goal at (5, 5)


In [3]:
from PIL import Image
from renderState import *

# Render some state of the level (here the initial state)
render_state(level_path=level_path, state=initial_state, output_path='twoagent')

# So the state looks like this in .txt format (what the computer uses):
print(initial_state)

# And the initial state looks like this in .png:
# You can either show the image in the notebook or open it in a new window
img = Image.open('twoagent.png')
img.show()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
State rendered and saved to twoagent.png
++++++++
+    2 +
+      +
+ 0    +
+      +
+ 1    +
+      +
++++++++


In [4]:
def render_plan(level_path, plan, strategy_name, heuristic_name, num_generated, elapsed_time, sol_length):

    str_plan = convert_plan_to_string(plan) #convert the plan to a string

    # this just makes sure that the meta information is displayed correctly in the visualization
    if strategy_name == 'greedy' or strategy_name == 'astar':
        strategy_name_pygame = strategy_name + ' w. ' + heuristic_name
    else:
        strategy_name_pygame = strategy_name
    
    subprocess.run(["python3", 
                    "renderMAvis.py", 
                    "--level", level_path, 
                    "--plan", str_plan, 
                    "--search_strategy", strategy_name_pygame, 
                    "--num_generated", str(num_generated), 
                    "--time_elapsed", str(elapsed_time), 
                    "--sol_length", str(sol_length)])

In [5]:
# Here is an example of a plan. A plan is a list of list of actions, where an action is an instance from searchclient.domains.hospital.actions
# The first axis of the list corresponds to a timestep in the plan. 
# Each element of the innermost list corresponds to an action of a different agent. 
# Check the documentation of graph_seach.py for more information!
hardcoded_plan = [[MoveAction("E")], [NoOpAction()], [MoveAction("W")]]

# Hardcode your solution here!
hardcoded_plan = [[MoveAction("E"), MoveAction("E"), NoOpAction()], [MoveAction("E"), MoveAction("S"), NoOpAction()], 
                  [MoveAction("E"), NoOpAction(), NoOpAction()], [MoveAction("S"), NoOpAction(), NoOpAction()], 
                  [MoveAction("S"), NoOpAction(), NoOpAction()]]

render_plan(level_path, hardcoded_plan, "Hardcoded sol", None, len(hardcoded_plan), "0.0", len(hardcoded_plan))

pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
Finished executing the plan


In [12]:
# Before running the search algorithm, we need to define the action set and the action library
# Use this library for pure pathfinding problems
action_library = DEFAULT_MAPF_ACTION_LIBRARY

# Use this library for sokoban-like problems (includes Push and Pull actions)
#action_library = DEFAULT_HOSPITAL_ACTION_LIBRARY

# Every agent will have the same action set
action_set = [action_library] * level.num_agents

# In order to run Graph-Search, we need to specify the initial state, action set, goal description, and frontier 

# If needed, we need to specify and fetch a heuristic function before initializing the frontier (informed search) 
# When adding new heuristics, remember to update the dictionary! 
# Use a string that matches the name of the heuristic function in the heuristics.py file
heuristic_name = "zero" 
heuristic = {
        'zero': HospitalZeroHeuristic,
        'goalcount': HospitalGoalCountHeuristics,
        'advanced': HospitalAdvancedHeuristics,
    }.get(heuristic_name, HospitalZeroHeuristic)() # make sure you understand what .get() does


# Finally, let's pick the search strategy and fetch the relevant frontier
strategy_name = "dfs" 
frontier = {
        'bfs': FrontierBFS,
        'dfs': FrontierDFS,
        'astar': lambda: FrontierAStar(heuristic),
        'greedy': lambda: FrontierGreedy(heuristic)
    }.get(strategy_name, FrontierBFS)() # make sure you understand what .get() does

In [13]:
# Now that we have defined the initial state, action set, goal description, and frontier, we can run the search algorithm
planning_success, plan, num_generated, elapsed_time = graph_search(initial_state, action_set, goal_description, frontier)

# The graph search function returns the following:
print('Planning successful:', planning_success)
print('Plan:', plan)
print('Solution length:', len(plan))
print('Number of states generated:', num_generated)
print('Elapsed time:', elapsed_time)

this is the frontier <searchclient.strategies.dfs.FrontierDFS object at 0x1138d75c0>


#Expanded:        0, #Frontier:        1, #Generated:        1, Time: 0,000 s, Memory: 79,05 MB


#Expanded:        1, #Frontier:       95, #Generated:       96, Time: 0,002 s, Memory: 79,48 MB


#Expanded:        2, #Frontier:      146, #Generated:      148, Time: 0,002 s, Memory: 79,48 MB


#Expanded:        3, #Frontier:      199, #Generated:      202, Time: 0,003 s, Memory: 79,48 MB


#Expanded:        4, #Frontier:      257, #Generated:      261, Time: 0,004 s, Memory: 79,53 MB


#Expanded:        5, #Frontier:      275, #Generated:      280, Time: 0,005 s, Memory: 79,53 MB


#Expanded:        6, #Frontier:      315, #Generated:      321, Time: 0,005 s, Memory: 79,53 MB


#Expanded:        7, #Frontier:      350, #Generated:      357, Time: 0,006 s, Memory: 79,53 MB


#Expanded:        8, #Frontier:      391, #Generated:      399, Time: 0,006 s, Memory: 79,53 MB


#Expanded:        9, #Frontier:      422, #Generated:      431, Time: 0,007 s, Memory: 79,53 MB


#Expanded:       10,

Planning successful: True
Plan: [(Move(W), Move(E), Move(E)), (Move(E), Move(S), Move(S)), (Move(E), Move(W), NoOp), (NoOp, Move(W), Move(N)), (Move(E), NoOp, Move(W)), (Move(S), Move(E), Move(E)), (Move(S), Move(E), NoOp), (NoOp, Move(E), Move(W)), (NoOp, Move(E), NoOp), (NoOp, Move(N), NoOp), (NoOp, Move(E), Move(S)), (Move(S), Move(N), Move(E)), (Move(E), Move(W), Move(W)), (Move(E), NoOp, Move(E)), (Move(N), Move(E), NoOp), (Move(W), Move(N), NoOp), (Move(N), NoOp, Move(N)), (Move(N), NoOp, Move(S)), (Move(W), NoOp, Move(N)), (Move(W), Move(S), Move(W)), (Move(W), NoOp, Move(S)), (Move(S), Move(W), Move(W)), (Move(S), NoOp, Move(W)), (Move(N), NoOp, Move(S)), (NoOp, Move(W), Move(W)), (Move(S), Move(N), NoOp), (Move(S), Move(W), NoOp), (NoOp, Move(N), Move(N)), (Move(E), Move(N), Move(W)), (Move(E), Move(W), Move(N)), (Move(N), Move(S), NoOp), (NoOp, Move(S), NoOp), (NoOp, Move(S), NoOp), (Move(S), Move(S), Move(E)), (Move(W), Move(W), Move(W)), (NoOp, Move(N), Move(E)), (Move(N), 

#Expanded:    2.288, #Frontier:   38.759, #Generated:   41.047, Time: 0,606 s, Memory: 126,38 MB


#Expanded:    2.289, #Frontier:   38.761, #Generated:   41.050, Time: 0,606 s, Memory: 126,38 MB


#Expanded:    2.290, #Frontier:   38.765, #Generated:   41.055, Time: 0,606 s, Memory: 126,38 MB


#Expanded:    2.291, #Frontier:   38.764, #Generated:   41.055, Time: 0,606 s, Memory: 126,38 MB


#Expanded:    2.292, #Frontier:   38.767, #Generated:   41.059, Time: 0,607 s, Memory: 126,38 MB


#Expanded:    2.293, #Frontier:   38.772, #Generated:   41.065, Time: 0,607 s, Memory: 126,39 MB


#Expanded:    2.294, #Frontier:   38.774, #Generated:   41.068, Time: 0,607 s, Memory: 126,39 MB


#Expanded:    2.295, #Frontier:   38.780, #Generated:   41.075, Time: 0,607 s, Memory: 126,39 MB


#Expanded:    2.296, #Frontier:   38.784, #Generated:   41.080, Time: 0,607 s, Memory: 126,39 MB


#Expanded:    2.297, #Frontier:   38.783, #Generated:   41.080, Time: 0,607 s, Memory: 126,39 MB


#Expanded:

In [14]:
render_plan(level_path, plan, strategy_name, heuristic, num_generated, elapsed_time, len(plan))

pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
Exiting...
